In [5]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!pip install ultralytics


In [9]:
import os
import time
from ultralytics import YOLO
import cv2

# ✅ 모델 경로 (.pt 파일까지)
model_path = "/content/drive/MyDrive/capstone/yolov5_2nd_version_train/yolov5n_ep100_bs16_img960/weights/best.pt"
image_dir = "/content/drive/MyDrive/capstone/capstone_dataset/capstone_project.v9i.yolov5pytorch/valid/images"

# ✅ 모델 로딩
model = YOLO(model_path)

# ✅ 이미지 목록
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.png'))]
image_paths = [os.path.join(image_dir, f) for f in image_files]

# ✅ FPS 측정
start = time.time()
for img_path in image_paths:
    model(img_path)
end = time.time()

# ✅ 계산
total = len(image_paths)
elapsed = end - start
fps = total / elapsed

# ✅ 평가
if fps >= 30:
    grade = "🏅 매우 우수 (CCTV 대응 가능)"
elif fps >= 15:
    grade = "👍 양호 (실시간 가능)"
elif fps >= 5:
    grade = "⚠️ 보통 (사후 분석용)"
else:
    grade = "❌ 느림 (업그레이드 필요)"

# ✅ 출력
print(f"\n📊 FPS 성능 평가 결과\n{'-'*40}")
print(f"총 이미지 수     : {total}")
print(f"총 소요 시간     : {elapsed:.2f}초")
print(f"평균 FPS         : {fps:.2f}")
print(f"성능 평가        : {grade}")
print(f"{'-'*40}")



image 1/1 /content/drive/MyDrive/capstone/capstone_dataset/capstone_project.v9i.yolov5pytorch/valid/images/frame_003900_jpg.rf.06fe6115c628ec86952551b3a9fa853c.jpg: 640x640 1 gown_on, 4 hairnet_ons, 6 persons, 7.1ms
Speed: 2.3ms preprocess, 7.1ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/capstone/capstone_dataset/capstone_project.v9i.yolov5pytorch/valid/images/frame_003540_jpg.rf.9e6ed26ccc834055586104e48af7b389.jpg: 640x640 3 gown_ons, 3 hairnet_ons, 4 persons, 7.9ms
Speed: 2.2ms preprocess, 7.9ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/capstone/capstone_dataset/capstone_project.v9i.yolov5pytorch/valid/images/frame_002100_jpg.rf.e99991537773641016ab9f0ef5f8cb59.jpg: 640x640 6 gown_ons, 9 hairnet_ons, 12 persons, 7.1ms
Speed: 1.6ms preprocess, 7.1ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/capstone/capstone_datase

In [14]:
import os
import cv2
import numpy as np
from ultralytics import YOLO

# ✅ 경로 설정
model_path = "/content/drive/MyDrive/capstone/yolov5_2nd_version_train/yolov5n_ep100_bs16_img960_t75v15t10/weights/best.pt"
image_dir = "/content/drive/MyDrive/capstone/capstone_dataset/capstone_project.v9i.yolov5pytorch/valid/images"
label_dir = "/content/drive/MyDrive/capstone/capstone_dataset/capstone_project.v9i.yolov5pytorch/valid/labels"

model = YOLO(model_path)

# ✅ IoU 함수
def compute_iou(box1, box2):
    x1, y1, x2, y2 = box1
    x1g, y1g, x2g, y2g = box2
    xi1, yi1 = max(x1, x1g), max(y1, y1g)
    xi2, yi2 = min(x2, x2g), min(y2, y2g)
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2g - x1g) * (y2g - y1g)
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area != 0 else 0

# ✅ 평가 루프
TP, FP, FN = 0, 0, 0
image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))]

for fname in image_files:
    img_path = os.path.join(image_dir, fname)
    label_path = os.path.join(label_dir, os.path.splitext(fname)[0] + '.txt')
    img = cv2.imread(img_path)
    h, w = img.shape[:2]

    # 🔍 예측 결과
    preds = model(img_path)[0].boxes.xyxy.cpu().numpy()

    # 🔍 GT 라벨
    gts = []
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            for line in f:
                cls, x, y, bw, bh = map(float, line.strip().split())
                x1 = (x - bw / 2) * w
                y1 = (y - bh / 2) * h
                x2 = (x + bw / 2) * w
                y2 = (y + bh / 2) * h
                gts.append([x1, y1, x2, y2])

    matched_gt = set()
    for pred in preds:
        found = False
        for i, gt in enumerate(gts):
            iou = compute_iou(pred[:4], gt)
            if iou >= 0.5 and i not in matched_gt:
                TP += 1
                matched_gt.add(i)
                found = True
                break
        if not found:
            FP += 1
    FN += len(gts) - len(matched_gt)

# ✅ Precision 계산
precision = TP / (TP + FP) if (TP + FP) > 0 else 0

# ✅ 출력
print(f"\n🎯 GT Precision 평가 결과\n{'-'*40}")
print(f"True Positive (TP) : {TP}")
print(f"False Positive (FP): {FP}")
print(f"False Negative (FN): {FN}")
print(f"Precision           : {precision:.4f}")
print(f"{'-'*40}")



image 1/1 /content/drive/MyDrive/capstone/capstone_dataset/capstone_project.v9i.yolov5pytorch/valid/images/frame_003900_jpg.rf.06fe6115c628ec86952551b3a9fa853c.jpg: 960x960 1 gown_on, 5 hairnet_ons, 4 persons, 11.8ms
Speed: 14.1ms preprocess, 11.8ms inference, 1.7ms postprocess per image at shape (1, 3, 960, 960)

image 1/1 /content/drive/MyDrive/capstone/capstone_dataset/capstone_project.v9i.yolov5pytorch/valid/images/frame_003540_jpg.rf.9e6ed26ccc834055586104e48af7b389.jpg: 960x960 3 gown_ons, 3 hairnet_ons, 4 persons, 11.8ms
Speed: 8.2ms preprocess, 11.8ms inference, 1.6ms postprocess per image at shape (1, 3, 960, 960)

image 1/1 /content/drive/MyDrive/capstone/capstone_dataset/capstone_project.v9i.yolov5pytorch/valid/images/frame_002100_jpg.rf.e99991537773641016ab9f0ef5f8cb59.jpg: 960x960 6 gown_ons, 9 hairnet_ons, 11 persons, 11.8ms
Speed: 8.4ms preprocess, 11.8ms inference, 1.6ms postprocess per image at shape (1, 3, 960, 960)

image 1/1 /content/drive/MyDrive/capstone/capstone